In [1]:
import torch
from torch import nn
from torch.autograd import Variable
from torch.nn import Module
import numpy as np
from torch.nn._functions.quantize import quantize
from torch.nn import functional as F


import torch.utils.data
import torchvision
from torchvision import datasets, transforms
import torch.optim as optim

from torch.nn.init import kaiming_normal
from torch.nn.init import xavier_normal

In [2]:
def kaiming_normal_quantized(tensor, bits, a=0, mode="fan_in"):
    kaiming_normal(tensor, a, mode)
    tensor = quantize(tensor, bits)
def xavier_normal_quantized(tensor, bits):
    xavier_normal(tensor)
    tensor = quantize(tensor, bits)

In [3]:
bits = (16, 12)
class Net(Module):
    def __init__(self):
        super(Net, self).__init__()
        self.q1 = nn.Quantize((16, 12),(16, 12))
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.q2 = nn.Quantize((16, 12),(16, 12))
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.q3 = nn.Quantize((16, 12),(16, 12))
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)
        
        for m in self.modules():
            if isinstance(m, nn.Conv2d) or isinstance(m, nn.Linear):
                xavier_normal_quantized(m.weight.data, bits)
                xavier_normal(m.weight.data)
    def forward(self, x):
        out = self.q1(x)
        out = F.relu(self.conv1(out))
        out = self.q2(out)
        out = F.max_pool2d(out,2)
        out = F.relu(self.conv2(out))
        out = self.q3(out)
        out = F.max_pool2d(out,2)
        
        out = out.view(out.size(0), -1)
        out = F.relu(self.fc1(out))
        out = F.relu(self.fc2(out))
        return F.log_softmax(out, dim=1)

In [4]:
bath_size = 100
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=bath_size, shuffle=True)

test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=bath_size, shuffle=True)

In [5]:
model = Net()

optimizer = optim.SGD(model.parameters(), lr=0.1, momentum=0.9)

def train(epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = Variable(data), Variable(target)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 100 == 0:
            print(model.conv1.weight.grad[0])
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.data[0]))

def test():
    model.eval()
    test_loss = 0
    correct = 0
    for data, target in test_loader:
        data, target = Variable(data, volatile=True), Variable(target)
        output = model(data)
        test_loss += F.nll_loss(output, target, size_average=False).data[0] # sum up batch loss
        pred = output.data.max(1, keepdim=True)[1] # get the index of the max log-probability
        correct += pred.eq(target.data.view_as(pred)).long().cpu().sum()

    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))


for epoch in range(1, 11):
    train(epoch)
    test()



(0 ,.,.) = 
1.00000e-02 *
   2.4569 -2.0266 -2.5841  1.4069  2.7186
   4.1677  0.1338 -0.2898  2.3578  4.4172
   1.7250 -1.0410 -0.5336  5.5086  7.0098
  -1.1682 -3.6599 -0.9773  4.7666  6.6331
  -4.7540 -5.7992 -2.4916  1.1972  3.0828
[torch.FloatTensor of size (1,5,5)]

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.306977


/Users/haonanchen/.pyenv/versions/miniconda3-latest/lib/python3.6/site-packages/ipykernel_launcher.py:18: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number



(0 ,.,.) = 
1.00000e-02 *
  -0.3479 -0.1004  0.0285 -0.2640 -0.5826
  -0.2852 -0.0164  0.5249  0.0044  0.0355
  -0.2061  0.4639  0.7667  0.2773 -0.4332
   0.2500  0.4691  0.9852  0.0681 -0.0685
   0.5938  0.9267  1.2739  0.1440 -0.0437
[torch.FloatTensor of size (1,5,5)]

Train Epoch: 1 [10000/60000 (17%)]	Loss: 1.974934

(0 ,.,.) = 
1.00000e-03 *
  -2.3910  0.9111  1.6438  2.7727  3.1413
  -4.0333  0.2224 -0.1294  1.6171  2.5406
  -4.3791 -0.1732 -0.1105  6.0641  2.8460
  -3.4519  1.0951  5.6765  6.0363  0.9501
  -5.4752 -0.7833  5.6615  4.8669 -0.1609
[torch.FloatTensor of size (1,5,5)]

Train Epoch: 1 [20000/60000 (33%)]	Loss: 1.840574


KeyboardInterrupt: 